In [2]:
%cd /home/jasper/Documents/PhD/Y0/conv_tasnet_source_sep

/home/jasper/Documents/PhD/Y0/conv_tasnet_source_sep


/home/jasper/miniforge3/envs/tasnet/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# !pip install asteroid

## Pre-Process



In [ ]:
!unzip /content/musdb18hq.zip

In [ ]:
!mkdir /content/musdb18_8000
!mkdir /content/musdb18_8000/train
!mkdir /content/musdb18_8000/test
!mkdir /content/musdb18_8000/train/dv_mix
!mkdir /content/musdb18_8000/train/drums
!mkdir /content/musdb18_8000/train/vocals
!mkdir /content/musdb18_8000/train/other
!mkdir /content/musdb18_8000/train/mixture
!mkdir /content/musdb18_8000/train/ins
!mkdir /content/musdb18_8000/train/bass

!mkdir /content/musdb18_8000/test/dv_mix
!mkdir /content/musdb18_8000/test/drums
!mkdir /content/musdb18_8000/test/vocals
!mkdir /content/musdb18_8000/test/other
!mkdir /content/musdb18_8000/test/mixture
!mkdir /content/musdb18_8000/test/ins
!mkdir /content/musdb18_8000/test/bass

In [ ]:
target_sr = 8000
batches=['train','test']

for batch in batches:
  for i, folder in enumerate(glob(f'/content/{batch}/*')):
    filename = folder.split('/')[-1].split('- ')[1].replace(' ', '_').replace("'", '').lower()
    print(f'{i:03}_{filename}')
    stems = []
    for f in glob(f'{folder}/*'):
      if f.split('/')[-1] == 'vocals.wav' or f.split('/')[-1] == 'drums.wav':
        print(f'mixing {f}')
        stem, sr = torchaudio.load(f)
        stems.append(stem)
    dvmix_data = torch.sum(torch.from_numpy(np.array(stems)), dim=0)
    dvmix_data = torchaudio.functional.resample(dvmix_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/dv_mix/{i:03}_{filename}_dvmix.wav', dvmix_data[0].unsqueeze(0),target_sr)

    print()

    stems = []
    for f in glob(f'{folder}/*'):
      if f.split('/')[-1] != 'mixture.wav' and f.split('/')[-1] != 'vocals.wav':
        print(f'mixing {f}')
        stem, sr = torchaudio.load(f)
        stems.append(stem)
    ins_data = torch.sum(torch.from_numpy(np.array(stems)), dim=0)
    ins_data = torchaudio.functional.resample(ins_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/ins/{i:03}_{filename}_ins.wav', ins_data[0].unsqueeze(0),target_sr)


    drums_data,sr = torchaudio.load(f'{folder}/drums.wav')
    drums_data = torchaudio.functional.resample(drums_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/drums/{i:03}_{filename}_drums.wav', drums_data[0].unsqueeze(0),target_sr)

    bass_data,sr = torchaudio.load(f'{folder}/bass.wav')
    bass_data = torchaudio.functional.resample(bass_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/bass/{i:03}_{filename}_bass.wav', bass_data[0].unsqueeze(0),target_sr)

    vocal_data,sr = torchaudio.load(f'{folder}/vocals.wav')
    vocal_data = torchaudio.functional.resample(vocal_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/vocals/{i:03}_{filename}_vocals.wav', vocal_data[0].unsqueeze(0),target_sr)

    other_data,sr = torchaudio.load(f'{folder}/other.wav')
    other_data = torchaudio.functional.resample(other_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/other/{i:03}_{filename}_other.wav', other_data[0].unsqueeze(0),target_sr)


    mixture_data,sr = torchaudio.load(f'{folder}/mixture.wav')
    mixture_data = torchaudio.functional.resample(mixture_data, sr, 8000)
    torchaudio.save(f'/content/musdb18_8000/{batch}/mixture/{i:03}_{filename}_mixture.wav', mixture_data[0].unsqueeze(0),target_sr)

    print()





## Create Dataset

In [3]:
import torch
import torchaudio
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import os

from glob import glob

class Datasets(Dataset):
    '''
       mix_path: file path of mixed audio (type: str)
       src_path: file path of ground truth sources (type: [str,...])
       sr: sample rate (type: int)
       chunk_size: framesize (type: int)
    '''

    def __init__(self, mix_path=None, src_path=None, sr=8000, chunk_size=32000):
        super(Datasets, self).__init__()
        self.mix_list = []
        for filename in glob(f'{mix_path}/*'):
          if filename[-4:] == '.wav':
            f,_ = torchaudio.load(filename)
            if f.shape[-1] > chunk_size:
              self.mix_list.append(filename)
        self.mix_list.sort()

        self.src_lists = []
        for p in src_path:
          src_list = []
          for filename in glob(f'{p}/*'):
            if filename[-4:] == '.wav':
              f,_ = torchaudio.load(filename)
              if f.shape[-1] > chunk_size:
                src_list.append(filename)
          src_list.sort()
          self.src_lists.append(src_list)

        print(f'loaded {len(self.mix_list)} mix\nloaded sources:')
        for l in self.src_lists:
          print(f'\t- {len(l)}')

        self.sr = sr
        self.chunk_size = chunk_size

    def __len__(self):
        return len(self.mix_list)

    def __getitem__(self, idx):

        mix, sr = torchaudio.load(self.mix_list[idx])
        assert sr == self.sr
        src = [torchaudio.load(src_list[idx])[0] for src_list in self.src_lists]

        assert mix.shape[-1] >= self.chunk_size
        if mix.shape[-1] == self.chunk_size:
            rand_start = 0
        else:
            rand_start = np.random.randint(0, mix.shape[-1] - self.chunk_size)

        return{
            'mix': mix[:,rand_start:rand_start+self.chunk_size],
            'src': [s[:,rand_start:rand_start+self.chunk_size] for s in src]
        }


In [4]:
train_data_kwargs = {
    'mix_path': './musdb18_8000/train/ins',
    'src_path': ['./musdb18_8000/train/bass','./musdb18_8000/train/drums', './musdb18_8000/train/other'],
    'sr': 8000,
    'chunk_size': 32000
}

val_data_kwargs = {
    'mix_path': './musdb18_8000/test/ins',
    'src_path': ['./musdb18_8000/test/bass','./musdb18_8000/test/drums', './musdb18_8000/test/other'],
    'sr': 8000,
    'chunk_size': 32000
}

loader_kwargs = {
    'batch_size': 2,
    'num_workers': 4,
    'shuffle': True,
    'drop_last': True
}

train_dataset = Datasets(**train_data_kwargs)
val_dataset = Datasets(**val_data_kwargs)

train_dataloader = DataLoader(train_dataset, **loader_kwargs)
val_dataloader = DataLoader(val_dataset, **loader_kwargs)


loaded 120 mix
loaded sources:
	- 120
	- 120
	- 120
loaded 30 mix
loaded sources:
	- 30
	- 30
	- 30


## Create Model

In [11]:
resume = 'runs/003/135.pt'

network_kwargs = {
    'in_channels': 1,
    'out_channels': 1,
    'masks_num': 3,
    'filters_num': 512,
    'filters_len': 16
}

tcn_kwargs = {
    'n_blocks': 8,
    'n_repeats': 3,
    'bn_chan': 128,
    'norm_type': 'gLN',
    'mask_act': 'prelu'
}

clip_norm = 200
log_period = 4
save_period = 15
num_epochs = 1000

checkpoint = './runs/003'
# checkpoint = '/content/Conv_TasNet_Source_Sep/t_003'

model = ConvTasNet(**network_kwargs, **tcn_kwargs)
cpt = torch.load('runs/003/135.pt', map_location='cpu')
model.load_state_dict(cpt['model_state_dict'])


<All keys matched successfully>

In [5]:
# resume = 'runs/003/135.pt'

network_kwargs = {
    'in_channels': 1,
    'out_channels': 1,
    'masks_num': 3,
    'filters_num': 512,
    'filters_len': 16,
    'tdcnpp': True,
    'enc_layer_num': 1, 
    'dec_layer_num': 1,
}

tcn_kwargs = {
    'n_blocks': 8,
    'n_repeats': 4,
    'bn_chan': 128,
    'norm_type': 'fgLN',
    'mask_act': 'prelu'
}

clip_norm = 200
log_period = 4
save_period = 50
num_epochs = 1000

checkpoint = './runs/014_mse_enc'

# changed to TDConvNetpp

# model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# cpt = torch.load('runs/003/135.pt', map_location='cpu')
# model.load_state_dict(cpt['model_state_dict'])

In [7]:
from asteroid.masknn.convolutional import TDConvNet, TDConvNetpp, DCUNetComplexEncoderBlock, DCUNetComplexDecoderBlock
from torch.nn import Conv1d, ConvTranspose1d, PReLU, Sequential
import torch

class Encoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len = 16, layer_num = 3) -> None:
      super().__init__()
      layers = []
      for _ in range(layer_num-1):
          layers.append(Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1))
          layers.append(PReLU())
      self.sequential = Sequential(
          Conv1d(in_c, out_c, filters_len, stride=filters_len//2), *layers
      )

  def forward(self, x):
      '''
        x: [Batch, in_channels, T]
        return: [Batch, filters_num, t]
      '''
      x = self.sequential(x)
      return x

class Decoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len = 16, layer_num = 3) -> None:
     super().__init__()
     layers = []
     for _ in range(layer_num-1):
         layers.append(ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1))
         layers.append(PReLU())
     self.sequential = Sequential(
         *layers,
         ConvTranspose1d(in_c, out_c, kernel_size=filters_len, stride=filters_len//2, bias=True)
     )

  def forward(self, x):
    '''
      x: [Batch, filters_num, t]
      return: [Batch, out_channels, T]
    '''
    x = self.sequential(x)
    return x

class ConvTasNet(torch.nn.Module):
  def __init__(self, in_channels, out_channels, masks_num, filters_num, filters_len = 16, tdcnpp = True, enc_layer_num=3, dec_layer_num=3,
               **tcn_kwargs) -> None:
    super().__init__()
    self.source_num = masks_num
    self.tdcnpp = tdcnpp
    self.filters_num = filters_num
    self.encoder = Encoder(in_channels, filters_num, filters_len, enc_layer_num)
    self.separation = TDConvNetpp(filters_num, masks_num, filters_num, **tcn_kwargs) if tdcnpp else TDConvNet(filters_num, masks_num, filters_num, **tcn_kwargs)
    self.decoder = Decoder(filters_num, out_channels, filters_len, dec_layer_num)
      
  def forward(self, x, return_mask = False):
    '''
      x: [Batch, Channels, T]
    '''
    assert x.dim() == 3
    
    w = self.encoder(x)
    # [Batch, filters_num, t]

    if self.tdcnpp:
        m, _ = self.separation(w)
    else:
        m = self.separation(w)
    # [Batch, mask_num, filters_num, t]

    assert m.dim() == 4
    assert m.shape[1] == self.source_num

    ws = w.unsqueeze(1).repeat([1,self.source_num,1,1])
    # [Batch, mask_num, filters_num, t]

    masked = m * ws
    # [Batch, mask_num, filters_num, t]

    masked = masked.unbind(dim=1)
    # [[Batch, filters_num, t], ...]: len = mask_num
    
    sources = [self.decoder(m) for m in masked]
    if return_mask:
        return sources, m
    else:
        return sources


In [8]:
# from asteroid.models.conv_tasnet import ConvTasNet
# from SI_SNR import si_snr_loss
# from torchmetrics.audio import ScaleInvariantSignalNoiseRatio

from torch.nn.utils import clip_grad_norm_


device = torch.device('cuda:0')

In [9]:
model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# model = ConvTasNet(3, out_chan=512, n_blocks=8, n_repeats=4, bn_chan=128, hid_chan=512, skip_chan=128, conv_kernel_size=3, norm_type='gLN', mask_act='prelu')

print(f'number of parameters: {sum(param.numel() for param in model.parameters())/10**6}M')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05)

if checkpoint and not os.path.exists(checkpoint):
    os.makedirs(checkpoint)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='min',
                                                       factor=0.5,
                                                       patience=25,
                                                       min_lr=1e-08)

mse = torch.nn.MSELoss().to(device)
# loss_fn = si_snr_loss
#si_snr = ScaleInvariantSignalNoiseRatio().to(device)

number of parameters: 6.77885M


In [10]:
model

ConvTasNet(
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Conv1d(1, 512, kernel_size=(16,), stride=(8,))
    )
  )
  (separation): TDConvNetpp(
    (bottleneck): Sequential(
      (0): FeatsGlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): FeatsGlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
  

In [11]:
import logging
import time

logger = logging.getLogger(os.path.join(checkpoint, "training.txt"))
logger.setLevel(logging.INFO)
# file or console
handler = logging.FileHandler(os.path.join(checkpoint, "training.txt"))
handler.setLevel(logging.INFO)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [12]:
def aggregate_loss(y, y_hat, loss_fn, device):
  N = len(y_hat)
  loss_all = torch.stack([loss_fn(y_hat[i], y[i].to(device)) for i in range(N)])
  # return - torch.sum(loss_all) / N
  return torch.sum(loss_all) / N

def mse_loss(y, y_hat, device):
  N = len(y_hat)
  loss_all = torch.stack([mse(y_hat[i], y[i].to(device)) for i in range(N)])
  # return - torch.sum(loss_all) / N
  return torch.sum(loss_all)

def train(model, train_loader, val_loader, optimizer, loss_fn, device, logger, sr = 8000):

  model.to(device)
  # loss_fn.to(device)

  total_loss = []
  total_val_loss = []

  torch.save({"epoch": 0,
              "model_state_dict": model.state_dict(),
              "optim_state_dict": optimizer.state_dict()
              },
              os.path.join(checkpoint, f'{0:03d}.pt'))

  logger.info('validating')

  cur_step = 0
  val_losses = []
  start = time.time()
  with torch.no_grad():
      for x in val_loader:
        y_hat = model(x['mix'].to(device))
        mse_loss_item = loss_fn(x['src'], y_hat, device)
        loss = mse_loss_item
        val_losses.append(loss.item())
        cur_step += 1
    
  duration = time.time() - start
  total_val_loss = sum(val_losses)/len(val_losses)
  logger.info(f'val_loss: {total_val_loss:.6f}, time: {duration}')
  for cur_epoch in range(num_epochs):
    logger.info('training')

    cur_step = 0
    losses = []
    mse_losses = []
    snr_losses = []
    start = time.time()
    for x in train_loader:

      optimizer.zero_grad()
      y_hat = model(x['mix'].to(device))
      # y_hat = torch.permute(y_hat, (1,0,2)).unsqueeze(2)
      # print(f'{y_hat[0].min()}, {y_hat[0].max()}, {y_hat[0].shape}, {len(y_hat)}')
      # loss = aggregate_loss(x['src'], y_hat, loss_fn, device)
      # snr_loss_item = loss_fn(y_hat, x, device)
      mse_loss_item = loss_fn(x['src'], y_hat, device)
      # loss = snr_loss_item + mse_loss_item
      loss = mse_loss_item
      loss.backward()

      clip_grad_norm_(model.parameters(), clip_norm)
      optimizer.step()

      losses.append(loss.item())
      mse_losses.append(mse_loss_item.item())
      # snr_losses.append(snr_loss_item.item())

      cur_step += 1

      if(len(losses)%log_period==0):
        mse_avg_loss = sum(mse_losses[-log_period:])/log_period
        # snr_avg_loss = sum(snr_losses[-log_period:])/log_period
        logger.info(f'epoch: {cur_epoch:03d}, step: {cur_step:03d}, avg_mse_loss: {mse_avg_loss:.6f}, lr: {optimizer.param_groups[0]["lr"]}')

    duration = time.time() - start
    logger.info(f'training time: {duration}')

    total_loss.append(sum(losses)/len(losses))
    logger.info(f'total_loss: {sum(losses)/len(losses)}')

    logger.info('validating')

    cur_step = 0
    val_losses = []
    start = time.time()
    with torch.no_grad():
      for x in val_loader:
        y_hat = model(x['mix'].to(device))
        # y_hat = torch.permute(y_hat, (1,0,2)).unsqueeze(2)
        # loss = aggregate_loss(x['src'], y_hat, loss_fn, device)
        # snr_loss_item = loss_fn(y_hat, x, device)
        mse_loss_item = loss_fn(x['src'], y_hat, device)
        # loss = snr_loss_item + mse_loss_item
        loss = mse_loss_item
        val_losses.append(loss.item())
        cur_step += 1

    duration = time.time() - start
    total_val_loss = sum(val_losses)/len(val_losses)
    logger.info(f'val_loss: {total_val_loss:.6f}, time: {duration}')

    scheduler.step(total_val_loss)

    if(cur_epoch%save_period==0):
      logger.info(f'save epoch {cur_epoch}')
      torch.save({"epoch": cur_epoch,
                  "model_state_dict": model.state_dict(),
                  "optim_state_dict": optimizer.state_dict()
                },
                os.path.join(checkpoint, f'{cur_epoch:03d}.pt'))

      for i, s in enumerate(y_hat):
        torchaudio.save(os.path.join(checkpoint, f'{cur_epoch:03d}_src_{i}.wav'),
                        s.reshape(-1).unsqueeze(0).cpu().detach(), sr)
      torchaudio.save(os.path.join(checkpoint, f'{cur_epoch:03d}_mix.wav'),
                        x['mix'].view(-1).unsqueeze(0).cpu().detach(), sr)

In [13]:
# train(model,train_dataloader,val_dataloader,optimizer,mse,device,logger)
train(model,train_dataloader,val_dataloader,optimizer,mse_loss,device,logger)


In [11]:
# resume = 'runs/003/135.pt'

network_kwargs = {
    'in_channels': 1,
    'out_channels': 1,
    'masks_num': 3,
    'filters_num': 512,
    'filters_len': 16
}

tcn_kwargs = {
    'n_blocks': 8,
    'n_repeats': 4,
    'bn_chan': 128,
    'norm_type': 'fgLN',
    'mask_act': 'prelu'
}

clip_norm = 200
log_period = 4
save_period = 50
num_epochs = 1000

checkpoint = './runs/011_mse_enc'

# changed to TDConvNetpp

# model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# cpt = torch.load('runs/003/135.pt', map_location='cpu')
# model.load_state_dict(cpt['model_state_dict'])

In [12]:
from asteroid.masknn.convolutional import TDConvNetpp


class Encoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
      super().__init__()
      self.sequential = Sequential(
          Conv1d(in_c, out_c, filters_len, stride=filters_len//2),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU(),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU(),
          # Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          # PReLU()
      )

  def forward(self, x):
      '''
        x: [Batch, in_channels, T]
        return: [Batch, filters_num, t]
      '''
      x = self.sequential(x)
      return x

class Decoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
     super().__init__()
     self.sequential = Sequential(
        #  ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
        #  PReLU(),
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, out_c, kernel_size=filters_len, stride=filters_len//2, bias=True)
     )

  def forward(self, x):
    '''
      x: [Batch, filters_num, t]
      return: [Batch, out_channels, T]
    '''
    x = self.sequential(x)
    return x

class ConvTasNet(torch.nn.Module):
  def __init__(self, in_channels, out_channels, masks_num, filters_num, filters_len = 16,
               **tcn_kwargs) -> None:
    super().__init__()
    self.source_num = masks_num
    # enc, dec = make_enc_dec('free',filters_num,filters_len,stride=filters_len//2)
    # self.encoder = enc
    # self.decoder = dec
    self.encoder = Encoder(in_channels, filters_num, filters_len)
    # self.encoder = DCUNetComplexEncoderBlock(in_channels, filters_num, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')
    self.separation = TDConvNetpp(filters_num, masks_num, filters_num, **tcn_kwargs)
    self.decoder = Decoder(filters_num, out_channels, filters_len)
    # self.decoder = DCUNetComplexDecoderBlock(filters_num, out_channels, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')

  def forward(self, x):
    '''
      x: [Batch, Channels, T]
    '''
    assert x.dim() == 3
    
    w = self.encoder(x)
    # [Batch, filters_num, t]

    m, _ = self.separation(w)
    # [Batch, mask_num, filters_num, t]

    assert m.dim() == 4
    assert m.shape[1] == self.source_num

    ws = w.unsqueeze(1).repeat([1,self.source_num,1,1])
    # [Batch, mask_num, filters_num, t]

    masked = m * ws
    # [Batch, mask_num, filters_num, t]

    masked = masked.unbind(dim=1)
    # [[Batch, filters_num, t], ...]: len = mask_num
    
    sources = [self.decoder(m) for m in masked]

    return sources


In [13]:
model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# model = ConvTasNet(3, out_chan=512, n_blocks=8, n_repeats=4, bn_chan=128, hid_chan=512, skip_chan=128, conv_kernel_size=3, norm_type='gLN', mask_act='prelu')

print(f'number of parameters: {sum(param.numel() for param in model.parameters())/10**6}M')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05)

if checkpoint and not os.path.exists(checkpoint):
    os.makedirs(checkpoint)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='min',
                                                       factor=0.5,
                                                       patience=25,
                                                       min_lr=1e-08)

mse = torch.nn.MSELoss().to(device)



number of parameters: 9.92663M


In [14]:
model

ConvTasNet(
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Conv1d(1, 512, kernel_size=(16,), stride=(8,))
      (1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): PReLU(num_parameters=1)
      (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): PReLU(num_parameters=1)
    )
  )
  (separation): TDConvNetpp(
    (bottleneck): Sequential(
      (0): FeatsGlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): FeatsGlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,)

In [15]:


logger = logging.getLogger(os.path.join(checkpoint, "training.txt"))
logger.setLevel(logging.INFO)
# file or console
handler = logging.FileHandler(os.path.join(checkpoint, "training.txt"))
handler.setLevel(logging.INFO)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [16]:
train(model,train_dataloader,val_dataloader,optimizer,mse_loss,device,logger)

In [3]:
# resume = 'runs/003/135.pt'

network_kwargs = {
    'in_channels': 1,
    'out_channels': 1,
    'masks_num': 3,
    'filters_num': 512,
    'filters_len': 16
}

tcn_kwargs = {
    'n_blocks': 8,
    'n_repeats': 4,
    'bn_chan': 128,
    'norm_type': 'fgLN',
    'mask_act': 'prelu'
}

clip_norm = 200
log_period = 4
save_period = 50
num_epochs = 1000

checkpoint = './runs/012_mse_enc'

# changed to TDConvNetpp, and 4 encoder layers

# model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# cpt = torch.load('runs/003/135.pt', map_location='cpu')
# model.load_state_dict(cpt['model_state_dict'])

In [7]:
from asteroid.masknn.convolutional import TDConvNetpp,DCUNetComplexEncoderBlock, DCUNetComplexDecoderBlock
from torch.nn import Conv1d, ConvTranspose1d, PReLU, Sequential


class Encoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
      super().__init__()
      self.sequential = Sequential(
          Conv1d(in_c, out_c, filters_len, stride=filters_len//2),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU(),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU(),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU()
      )

  def forward(self, x):
      '''
        x: [Batch, in_channels, T]
        return: [Batch, filters_num, t]
      '''
      x = self.sequential(x)
      return x

class Decoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
     super().__init__()
     self.sequential = Sequential(
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, out_c, kernel_size=filters_len, stride=filters_len//2, bias=True)
     )

  def forward(self, x):
    '''
      x: [Batch, filters_num, t]
      return: [Batch, out_channels, T]
    '''
    x = self.sequential(x)
    return x

class ConvTasNet(torch.nn.Module):
  def __init__(self, in_channels, out_channels, masks_num, filters_num, filters_len = 16,
               **tcn_kwargs) -> None:
    super().__init__()
    self.source_num = masks_num
    # enc, dec = make_enc_dec('free',filters_num,filters_len,stride=filters_len//2)
    # self.encoder = enc
    # self.decoder = dec
    self.encoder = Encoder(in_channels, filters_num, filters_len)
    # self.encoder = DCUNetComplexEncoderBlock(in_channels, filters_num, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')
    self.separation = TDConvNetpp(filters_num, masks_num, filters_num, **tcn_kwargs)
    self.decoder = Decoder(filters_num, out_channels, filters_len)
    # self.decoder = DCUNetComplexDecoderBlock(filters_num, out_channels, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')

  def forward(self, x):
    '''
      x: [Batch, Channels, T]
    '''
    assert x.dim() == 3
    
    w = self.encoder(x)
    # [Batch, filters_num, t]

    m, _ = self.separation(w)
    # [Batch, mask_num, filters_num, t]

    assert m.dim() == 4
    assert m.shape[1] == self.source_num

    ws = w.unsqueeze(1).repeat([1,self.source_num,1,1])
    # [Batch, mask_num, filters_num, t]

    masked = m * ws
    # [Batch, mask_num, filters_num, t]

    masked = masked.unbind(dim=1)
    # [[Batch, filters_num, t], ...]: len = mask_num
    
    sources = [self.decoder(m) for m in masked]

    return sources


In [5]:
# from asteroid.models.conv_tasnet import ConvTasNet
# from SI_SNR import si_snr_loss
# from torchmetrics.audio import ScaleInvariantSignalNoiseRatio

from torch.nn.utils import clip_grad_norm_


device = torch.device('cuda:0')

In [8]:
model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# model = ConvTasNet(3, out_chan=512, n_blocks=8, n_repeats=4, bn_chan=128, hid_chan=512, skip_chan=128, conv_kernel_size=3, norm_type='gLN', mask_act='prelu')

print(f'number of parameters: {sum(param.numel() for param in model.parameters())/10**6}M')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05)

if checkpoint and not os.path.exists(checkpoint):
    os.makedirs(checkpoint)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='min',
                                                       factor=0.5,
                                                       patience=25,
                                                       min_lr=1e-08)

mse = torch.nn.MSELoss().to(device)
# loss_fn = si_snr_loss
#si_snr = ScaleInvariantSignalNoiseRatio().to(device)

number of parameters: 11.50052M


In [10]:
model

ConvTasNet(
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Conv1d(1, 512, kernel_size=(16,), stride=(8,))
      (1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): PReLU(num_parameters=1)
      (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): PReLU(num_parameters=1)
      (5): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (6): PReLU(num_parameters=1)
    )
  )
  (separation): TDConvNetpp(
    (bottleneck): Sequential(
      (0): FeatsGlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): FeatsGlobLN()
        )
        (res_c

In [11]:
import logging
import time

logger = logging.getLogger(os.path.join(checkpoint, "training.txt"))
logger.setLevel(logging.INFO)
# file or console
handler = logging.FileHandler(os.path.join(checkpoint, "training.txt"))
handler.setLevel(logging.INFO)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [12]:
train(model,train_dataloader,val_dataloader,optimizer,mse_loss,device,logger)

In [13]:
# resume = 'runs/003/135.pt'

network_kwargs = {
    'in_channels': 1,
    'out_channels': 1,
    'masks_num': 3,
    'filters_num': 512,
    'filters_len': 16
}

tcn_kwargs = {
    'n_blocks': 8,
    'n_repeats': 4,
    'bn_chan': 128,
    'norm_type': 'fgLN',
    'mask_act': 'prelu'
}

clip_norm = 200
log_period = 4
save_period = 50
num_epochs = 1000

checkpoint = './runs/013_mse_enc'

# changed to TDConvNetpp, and 2 encoder layers

# model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# cpt = torch.load('runs/003/135.pt', map_location='cpu')
# model.load_state_dict(cpt['model_state_dict'])

In [14]:
from asteroid.masknn.convolutional import TDConvNetpp,DCUNetComplexEncoderBlock, DCUNetComplexDecoderBlock
from torch.nn import Conv1d, ConvTranspose1d, PReLU, Sequential


class Encoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
      super().__init__()
      self.sequential = Sequential(
          Conv1d(in_c, out_c, filters_len, stride=filters_len//2),
          Conv1d(out_c, out_c, kernel_size=3, stride=1, padding=1),
          PReLU(),
      )

  def forward(self, x):
      '''
        x: [Batch, in_channels, T]
        return: [Batch, filters_num, t]
      '''
      x = self.sequential(x)
      return x

class Decoder(torch.nn.Module):
  def __init__(self, in_c, out_c, filters_len) -> None:
     super().__init__()
     self.sequential = Sequential(
         ConvTranspose1d(in_c, in_c, kernel_size=3, stride=1, padding=1),
         PReLU(),
         ConvTranspose1d(in_c, out_c, kernel_size=filters_len, stride=filters_len//2, bias=True)
     )

  def forward(self, x):
    '''
      x: [Batch, filters_num, t]
      return: [Batch, out_channels, T]
    '''
    x = self.sequential(x)
    return x

class ConvTasNet(torch.nn.Module):
  def __init__(self, in_channels, out_channels, masks_num, filters_num, filters_len = 16,
               **tcn_kwargs) -> None:
    super().__init__()
    self.source_num = masks_num
    # enc, dec = make_enc_dec('free',filters_num,filters_len,stride=filters_len//2)
    # self.encoder = enc
    # self.decoder = dec
    self.encoder = Encoder(in_channels, filters_num, filters_len)
    # self.encoder = DCUNetComplexEncoderBlock(in_channels, filters_num, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')
    self.separation = TDConvNetpp(filters_num, masks_num, filters_num, **tcn_kwargs)
    self.decoder = Decoder(filters_num, out_channels, filters_len)
    # self.decoder = DCUNetComplexDecoderBlock(filters_num, out_channels, filters_len, filters_len//2, padding = 1, norm_type='gLN', activation='prelu')

  def forward(self, x):
    '''
      x: [Batch, Channels, T]
    '''
    assert x.dim() == 3
    
    w = self.encoder(x)
    # [Batch, filters_num, t]

    m, _ = self.separation(w)
    # [Batch, mask_num, filters_num, t]

    assert m.dim() == 4
    assert m.shape[1] == self.source_num

    ws = w.unsqueeze(1).repeat([1,self.source_num,1,1])
    # [Batch, mask_num, filters_num, t]

    masked = m * ws
    # [Batch, mask_num, filters_num, t]

    masked = masked.unbind(dim=1)
    # [[Batch, filters_num, t], ...]: len = mask_num
    
    sources = [self.decoder(m) for m in masked]

    return sources


In [15]:
model = ConvTasNet(**network_kwargs, **tcn_kwargs)
# model = ConvTasNet(3, out_chan=512, n_blocks=8, n_repeats=4, bn_chan=128, hid_chan=512, skip_chan=128, conv_kernel_size=3, norm_type='gLN', mask_act='prelu')

print(f'number of parameters: {sum(param.numel() for param in model.parameters())/10**6}M')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05)

if checkpoint and not os.path.exists(checkpoint):
    os.makedirs(checkpoint)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='min',
                                                       factor=0.5,
                                                       patience=25,
                                                       min_lr=1e-08)

mse = torch.nn.MSELoss().to(device)
# loss_fn = si_snr_loss
#si_snr = ScaleInvariantSignalNoiseRatio().to(device)

number of parameters: 8.35274M


In [16]:
model

ConvTasNet(
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Conv1d(1, 512, kernel_size=(16,), stride=(8,))
      (1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): PReLU(num_parameters=1)
    )
  )
  (separation): TDConvNetpp(
    (bottleneck): Sequential(
      (0): FeatsGlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): FeatsGlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128

In [17]:


logger = logging.getLogger(os.path.join(checkpoint, "training.txt"))
logger.setLevel(logging.INFO)
# file or console
handler = logging.FileHandler(os.path.join(checkpoint, "training.txt"))
handler.setLevel(logging.INFO)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [18]:
train(model,train_dataloader,val_dataloader,optimizer,mse_loss,device,logger)